### Scrape List of S&P 500 Companies From Wikipedia

In [1]:
import pandas as pd

In [2]:
# Scrape the S&P 500 companies from wikipedia using pandas
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

df = pd.read_html(url)
s_p_500_df = df[0]
s_p_500_df.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [3]:
# Export the scraped values to a csv file
s_p_500_df.to_csv('S&P500_data_scraped_wikipedia.csv', index=False)
s_p_500_df.shape

(505, 9)

In [4]:
# Bring in csv file which includes the lat and lon of all publicly traded companies
all_sec_path = 'clean_sec_companies_with_tickers_coord.csv'
all_sec_df = pd.read_csv(all_sec_path)
all_sec_df.head()

,ticker,cik,name,country,state,city,zipcode,lat,lon
0,AIR,1750,AAR CORP,US,IL,WOOD DALE,60191,41.985680,-87.97991
1,WDDD,1961,WORLDS INC,US,MA,BROOKLINE,2445,42.336193,-71.12950
2,ACU,2098,ACME UNITED CORP,US,CT,FAIRFIELD,6824,41.147858,-73.25259
3,AE,2178,"ADAMS RESOURCES & ENERGY, INC.",US,TX,HOUSTON,77027,29.751370,-95.45202
4,BKTI,2186,BK Technologies Corp,US,FL,WEST MELBOURNE,32904,28.096567,-80.68530


In [5]:
# Merge the lat and long values into the S&P 500 companies
sp_comb_df = pd.merge(s_p_500_df, all_sec_df, how="left", left_on=['Symbol', 'CIK'], right_on=['ticker', 'cik'])

In [6]:
# Filter for only the US companies in the S&P 500
mask1 = sp_comb_df['country'] == 'US'
sp_comb_us_df = sp_comb_df[mask1]

In [7]:
# Drop columns that are not necessary
sp_comb_us_df = sp_comb_us_df.drop(columns=['SEC filings', 'name', 'cik', 'ticker', 'Founded', 'Date first added', 'Headquarters Location'], axis=1)
sp_comb_us_df['index']='S&P 500'
sp_comb_us_df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,CIK,country,state,city,zipcode,lat,lon,index
0,MMM,3M Company,Industrials,Industrial Conglomerates,66740,US,MN,ST PAUL,55144-1000,44.943700,-93.094300,S&P 500
2,ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,1551152,US,IL,NORTH CHICAGO,60064,42.319400,-87.856100,S&P 500
3,ABMD,ABIOMED Inc,Health Care,Health Care Equipment,815094,US,MA,DANVERS,1923,42.576073,-70.916016,S&P 500
5,ATVI,Activision Blizzard,Communication Services,Interactive Home Entertainment,718877,US,CA,SANTA MONICA,90405,34.020350,-118.453120,S&P 500
6,ADBE,Adobe Inc.,Information Technology,Application Software,796343,US,CA,SAN JOSE,95110-2704,37.333300,-121.900000,S&P 500


In [8]:
# Check to see how many US companies are included in the S&P 500
sp_comb_us_df.shape

(467, 12)

In [9]:
# Convert the CIK value from an integer to a string object
sp_comb_us_df['CIK'] = sp_comb_us_df['CIK'].astype('str')

### Scrape List of Nasdaq Companies From Wikipedia

In [11]:
# Scrape the S&P 500 companies from wikipedia using pandas
nasdaq_url = 'https://en.wikipedia.org/wiki/NASDAQ-100#Components'

df = pd.read_html(nasdaq_url)[3]
nasdaq_df = df
# Reorder the columns
nasdaq_df = nasdaq_df[['Ticker', 'Company', 'GICS Sector', 'GICS Sub-Industry']]
nasdaq_df.head()

,Ticker,Company,GICS Sector,GICS Sub-Industry
0,ATVI,Activision Blizzard,Communication Services,Interactive Home Entertainment
1,ADBE,Adobe Inc.,Information Technology,Application Software
2,AMD,Advanced Micro Devices,Information Technology,Semiconductors
3,ALXN,Alexion Pharmaceuticals,Health Care,Pharmaceuticals
4,ALGN,Align Technology,Health Care,Health Care Supplies


In [12]:
nasdaq_df.shape

(103, 4)

In [13]:
# Merge the scraped NASDAQ data with the SEC data to include lat lon data
nasdaq_ext_df = pd.merge(nasdaq_df, all_sec_df, how="left", left_on="Ticker", right_on="ticker")
nasdaq_ext_df.head()

,Ticker,Company,GICS Sector,GICS Sub-Industry,ticker,cik,name,country,state,city,zipcode,lat,lon
0,ATVI,Activision Blizzard,Communication Services,Interactive Home Entertainment,ATVI,718877.0,"Activision Blizzard, Inc.",US,CA,SANTA MONICA,90405,34.020350,-118.45312
1,ADBE,Adobe Inc.,Information Technology,Application Software,ADBE,796343.0,ADOBE INC.,US,CA,SAN JOSE,95110-2704,37.333300,-121.90000
2,AMD,Advanced Micro Devices,Information Technology,Semiconductors,AMD,2488.0,ADVANCED MICRO DEVICES INC,US,CA,SANTA CLARA,95054,37.382294,-121.97213
3,ALXN,Alexion Pharmaceuticals,Health Care,Pharmaceuticals,ALXN,899866.0,"ALEXION PHARMACEUTICALS, INC.",US,MA,BOSTON,2210,42.350197,-71.04221
4,ALGN,Align Technology,Health Care,Health Care Supplies,ALGN,1097149.0,ALIGN TECHNOLOGY INC,US,CA,SAN JOSE,95134,37.389860,-121.93604


In [14]:
# Drop unnecessary columns from the nasdaq dataframe and add the NASDAQ index
nasdaq_ext_df.drop(columns=(['ticker', 'name']), inplace=True)
nasdaq_ext_df['index'] = nasdaq_ext_df['index'] = "NASDAQ"

In [15]:
nasdaq_ext_df.shape

(103, 12)

In [16]:
# Rename the column header so the S&P and Nasdaq columns are the same
sp_comb_us_df.rename(columns = {'Symbol':'ticker', 'Security':'company', 'GICS Sector':'gics_sector', \
                                'GICS Sub-Industry':'gics_sub_industry', 'CIK':'cik'}, inplace=True)


In [17]:
# Rename the column header so the S&P and Nasdaq columns are the same
nasdaq_ext_df.rename(columns = {'Ticker':'ticker', 'Company':'company', 'GICS Sector':'gics_sector', \
                                'GICS Sub-Industry':'gics_sub_industry'}, inplace=True)


In [18]:
# Filter NASDAQ companies for US companies only
mask1=nasdaq_ext_df['country'] == 'US'
nasdaq_us_df = nasdaq_ext_df[mask1]
nasdaq_us_df.head()

,ticker,company,gics_sector,gics_sub_industry,cik,country,state,city,zipcode,lat,lon,index
0,ATVI,Activision Blizzard,Communication Services,Interactive Home Entertainment,718877.0,US,CA,SANTA MONICA,90405,34.020350,-118.45312,NASDAQ
1,ADBE,Adobe Inc.,Information Technology,Application Software,796343.0,US,CA,SAN JOSE,95110-2704,37.333300,-121.90000,NASDAQ
2,AMD,Advanced Micro Devices,Information Technology,Semiconductors,2488.0,US,CA,SANTA CLARA,95054,37.382294,-121.97213,NASDAQ
3,ALXN,Alexion Pharmaceuticals,Health Care,Pharmaceuticals,899866.0,US,MA,BOSTON,2210,42.350197,-71.04221,NASDAQ
4,ALGN,Align Technology,Health Care,Health Care Supplies,1097149.0,US,CA,SAN JOSE,95134,37.389860,-121.93604,NASDAQ


In [19]:
nasdaq_us_df.shape

(91, 12)

### Combine S&P500 and NASDAQ data

In [20]:
combined_us_stocks_df = pd.concat([sp_comb_us_df, nasdaq_us_df], ignore_index=True)
combined_us_stocks_df.head()

,ticker,company,gics_sector,gics_sub_industry,cik,country,state,city,zipcode,lat,lon,index
0,MMM,3M Company,Industrials,Industrial Conglomerates,66740,US,MN,ST PAUL,55144-1000,44.943700,-93.094300,S&P 500
1,ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,1551152,US,IL,NORTH CHICAGO,60064,42.319400,-87.856100,S&P 500
2,ABMD,ABIOMED Inc,Health Care,Health Care Equipment,815094,US,MA,DANVERS,1923,42.576073,-70.916016,S&P 500
3,ATVI,Activision Blizzard,Communication Services,Interactive Home Entertainment,718877,US,CA,SANTA MONICA,90405,34.020350,-118.453120,S&P 500
4,ADBE,Adobe Inc.,Information Technology,Application Software,796343,US,CA,SAN JOSE,95110-2704,37.333300,-121.900000,S&P 500


In [21]:
# Function to convert the uppercase city names to title case
combined_us_stocks_df['town'] = ''

def title_case(str):
    conv_string = str.title()
    return conv_string

combined_us_stocks_df['town'] = combined_us_stocks_df['city'].apply(title_case)

combined_us_stocks_df['city'] = combined_us_stocks_df['town']
combined_us_stocks_df.drop(columns=['town'], inplace=True)

combined_us_stocks_df.head()    

,ticker,company,gics_sector,gics_sub_industry,cik,country,state,city,zipcode,lat,lon,index
0,MMM,3M Company,Industrials,Industrial Conglomerates,66740,US,MN,St Paul,55144-1000,44.943700,-93.094300,S&P 500
1,ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,1551152,US,IL,North Chicago,60064,42.319400,-87.856100,S&P 500
2,ABMD,ABIOMED Inc,Health Care,Health Care Equipment,815094,US,MA,Danvers,1923,42.576073,-70.916016,S&P 500
3,ATVI,Activision Blizzard,Communication Services,Interactive Home Entertainment,718877,US,CA,Santa Monica,90405,34.020350,-118.453120,S&P 500
4,ADBE,Adobe Inc.,Information Technology,Application Software,796343,US,CA,San Jose,95110-2704,37.333300,-121.900000,S&P 500


In [24]:
combined_us_stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558 entries, 0 to 557
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ticker             558 non-null    object 
 1   company            558 non-null    object 
 2   gics_sector        558 non-null    object 
 3   gics_sub_industry  558 non-null    object 
 4   cik                558 non-null    object 
 5   country            558 non-null    object 
 6   state              558 non-null    object 
 7   city               558 non-null    object 
 8   zipcode            558 non-null    object 
 9   lat                558 non-null    float64
 10  lon                558 non-null    float64
 11  index              558 non-null    object 
dtypes: float64(2), object(10)
memory usage: 52.4+ KB


In [23]:
# Export the combined US stock data to a csv file
combined_us_stocks_df.to_csv('Combined_US_Stocks.csv', index=False)